# Pipeline with scikit-learn
- Cross Validate the whole process, including pre-processing and the model
- Grid Search parameters for pre-processing steps and model building

---

# Workflow Summary

### 1. Load Data:

In [26]:
import pandas as pd

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [44]:
# Load Train and Test Data
df = pd.read_csv("Data/titanic_train.csv")
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]
X_train = df.drop('Survived', axis='columns')
y_train = df.Survived

X_test = pd.read_csv("Data/titanic_test.csv")
X_test = df_test.loc[:, ['Pclass', 'Sex', 'Embarked']]

### 2. Pre-process with Column Transformer and Initiate the Model

In [45]:
# Apply OneHotEncoder to Sex and Embarked columns and skip the remainder
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']),
    remainder='passthrough')

# Initiate Logistic Regression Model
model_logreg = LogisticRegression(solver='lbfgs')

### 3. Define the Pipeline

In [46]:
pipe = make_pipeline(column_trans, model_logreg)

### 4. Cross Validate the Pipeline

In [47]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

### 5. Fit the Model (Pipeline) and Predict on New Test Data

In [48]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
y_pred[0:10]

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0])

---

# Detailed Breakdown

---

## Load Data and Modules
- Using Titanic Dataset

In [3]:
df = pd.read_csv("Data/titanic_train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Look at any missing data

In [5]:
df.shape

(891, 12)

In [4]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

#### Select Feature Columns and exclude missing rows

In [6]:
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]
df.shape

(889, 4)

In [7]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


#### Split Data into Features and Labels
- Determine the Null Accuracy: Predicting the most frequent class

In [20]:
X = df.drop('Survived', axis='columns')
y = df.Survived
print(f"X: {X.shape}")
print(f"y: {y.shape}")

X: (889, 3)
y: (889,)


In [12]:
# Null Accuracy: 61.7%
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

---

## One Hot Encoder
- Encoding Unordered Categorical Features, also known as Dummy Encoding in pandas
- sklearn `OneHotEncoder` vs pandas `getDummies` 
    - Does not affect the original dataframe
    - No need to manually dummy encode Test Data
    - Resolves problem with Test Data having different categories than Training Data

In [17]:
ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(df[['Embarked']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [18]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

---

## Define a Pipeline with One Hot Encoder and Column Transformer
- Use Column Transformer when there are Features that need different pre-processesing
- Apply OneHotEncoder to Sex and Embarked columns only

In [22]:
# Apply OneHotEncoder to Sex and Embarked columns and skip the remainder
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']),
    remainder='passthrough')

In [23]:
column_trans

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('onehotencoder',
                                 OneHotEncoder(categories='auto', drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=True),
                                 ['Sex', 'Embarked'])],
                  verbose=False)

#### Apply to the Training Data
- First 2 columns refer to One Hot Encoded *Sex* column
- Next 3 columns refer to the One Hot Encoded *Embarked* column
- Final column is the **passthrough** of the *Pclass* column

In [24]:
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

#### Initiate the Model

In [27]:
model_logreg = LogisticRegression(solver='lbfgs')

#### Define the Pipeline with Column Transformer and the Model

In [28]:
pipe = make_pipeline(column_trans, model_logreg)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['Sex', 'Embarked'])],
                                   verbose=False)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                         

---

## Cross Validate the Pipeline
- Cross Validate entire pipeline to evaluate the Model

In [29]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

#### Predict on new Test Data

In [39]:
X_test = pd.read_csv("Data/titanic_test.csv")
X_test = df_test.loc[:, ['Pclass', 'Sex', 'Embarked']]
X_test.shape

(418, 3)

In [40]:
pipe.fit(X, y)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['Sex', 'Embarked'])],
                                   verbose=False)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                         

In [43]:
y_pred = pipe.predict(X_test)
y_pred[0:10]

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0])

---